In [1]:
import sys, os
sys.path.append("../iotools/")
import readgtf
from collections import defaultdict
import pandas as pd
import argparse
import re

In [2]:
def filter_donors(df, donors):
    donor_list = df.columns
    common  = [x for x in donors if x in donor_list]
    print("{:d} donors remained from {:d}".format(len(common), len(donor_list)))
    return df[common]

def filter_rows(df, genedict):
    gx_gene_list = df.index
    common  = [genedict[x] for x in gx_gene_list]
    print("{:d} genes remained from {:d}".format(sum(common), len(gx_gene_list)))
    return df[common]


# gx_file = "/cbscratch/franco/trans-eqtl/new_preprocess_feb2020_freeze/gtex_v8/expression/tpms/wb_tpms_qcfilter.txt.protein_coding_lncRNA_filtered"
gx_file = "/cbscratch/franco/datasets/gtex_v8/expression/TPMs_phASER_GTEx_v8.matrix.txt"
gtf_file= "/cbscratch/franco/datasets/GENCODE/gencode.v26.annotation.gtf.gz"

print("Reading GENCODE file")
gene_info = readgtf.gencode_v12(gtf_file, trim=False)

Reading GENCODE file


In [3]:
gene_dict = defaultdict(lambda: False)
for g in gene_info:
    gene_dict[g.ensembl_id] = g.name

gx_df = pd.read_table(gx_file, sep="\t", header=0, index_col=0)


# new_gx_df = filter_rows(gx_df, gene_dict)

# outdir = os.path.dirname(os.path.realpath(args.out_file))
# if not os.path.exists(outdir):
#     os.makedirs(outdir)
# sorted_gx_df.to_csv(args.out_file, doublequote=False, sep="\t")

In [4]:
gene_names = [gene_dict[n] for n in gx_df.index]
gx_df.index = gene_names

In [10]:
ix = gx_df.index != False
final_gene_names = list(final_gx.index)
final_gx = gx_df[ix]

(53288, 15253)

In [16]:
dupl = set() # create empty set

# loop trough the elements inside the list
for e in final_gene_names:
    if final_gene_names.count(e) > 1:
        dupl.add(e)

In [17]:
dupl

{'C2orf61',
 'CYB561D2',
 'DNAJC9-AS1',
 'ELFN2',
 'GOLGA8M',
 'LINC00484',
 'LINC01115',
 'LINC01238',
 'LINC01297',
 'LINC01422',
 'LINC01481',
 'LINC01598',
 'LYNX1',
 'MAL2',
 'MIR3179-3',
 'MIR3180-4',
 'PROX1-AS1',
 'RAET1E-AS1',
 'RGS5',
 'SPATA13'}

In [21]:
final_gx_rmv_dup = final_gx[~final_gx.index.isin(dupl)]
final_gx_rmv_dup.shape

(33588, 15253)

In [28]:
import numpy as np
ix_zeros = np.all(final_gx_rmv_dup == 0, axis=1)
final_gx_clean = final_gx_rmv_dup[~ix_zeros]

In [31]:
final_gx_clean.shape

(32416, 15253)

In [32]:
outfile = gx_file + ".gene_names"
# gx_df.to_csv(outfile, sep="\t")
final_gx_clean.to_csv(outfile, sep="\t")